<a href="https://colab.research.google.com/github/MET-CS-777/assignment-1t/blob/main/Assignment_1_Helper_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 helper code
**NOTE**

- You can use this Colab Notebook only to quickly start with Spark.
- This code is only for an intro to the Spark platform and can not be used as an assignment 1 submission. 
- The assignment must be done using a python script that can run on the Google Cloud Platform. Use the .py template to submit your assignment.

In [15]:
# Install PySpark on Colab Vrtula Machine
!pip3 install pyspark

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext("local")
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/27 20:49:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#Exception Handling and removing wrong datalines
def isfloat(value):
    try:
        float(value)
        return True
 
    except:
         return False

In [5]:
#For example, remove lines if they don’t have 16 values and 
# checking if the trip distance and fare amount is a float number
# checking if the trip duration is more than a minute, trip distance is more than 0.1 miles, 
# fare amount and total amount are more than 0.1 dollars
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[4])> 60 and float(p[5])>0.10 and float(p[11])> 0.10 and float(p[16])> 0.10):
                return p



In [11]:
# Upload the data to Colab Virtual Machine
# The data is downoladed from the web and stored localy. The wget is a linux command to get and download files from web.
! wget -q https://storage.googleapis.com/met-cs-777-data/taxi-data-sorted-small.csv.bz2
# At this point you have to upload your data to google colab local file system

In [13]:
# Set your file path here 
path="/Users/pauljiang/CS777/Assignment1/"
testFile= path + "taxi-data-sorted-small.csv.bz2"

# Read the data in Spark DataFrame
testDataFrame = spark.read.format('csv').options(header='false', inferSchema='true',  sep =",").load(testFile)
testDataFrame.show()

+--------------------+--------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+----+----+----+----+----+----+----+
|                 _c0|                 _c1|                _c2|                _c3|_c4| _c5|       _c6|      _c7|       _c8|      _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|
+--------------------+--------------------+-------------------+-------------------+---+----+----------+---------+----------+---------+----+----+----+----+----+----+----+
|07290D3599E7A0D62...|E7750A37CAB07D0DF...|2013-01-01 00:00:00|2013-01-01 00:02:00|120|0.44|-73.956528|40.716976| -73.96244|40.715008| CSH| 3.5| 0.5| 0.5| 0.0| 0.0| 4.5|
|22D70BF00EEB0ADC8...|3FF2709163DE7036F...|2013-01-01 00:02:00|2013-01-01 00:02:00|  0| 0.0|       0.0|      0.0|       0.0|      0.0| CSH|27.0| 0.0| 0.5| 0.0| 0.0|27.5|
|0EC22AAF491A8BD91...|778C92B26AE78A9EB...|2013-01-01 00:01:00|2013-01-01 00:03:00|120|0.71|-73.973145|40.752827|-73.965897|40.760445| CSH| 4.0| 0.5| 

In [14]:
# Convert data from DataFrame to RDD
testRDD = testDataFrame.rdd.map(tuple)
testRDD.take(1)

[('07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  datetime.datetime(2013, 1, 1, 0, 0),
  datetime.datetime(2013, 1, 1, 0, 2),
  120,
  0.44,
  -73.956528,
  40.716976,
  -73.96244,
  40.715008,
  'CSH',
  3.5,
  0.5,
  0.5,
  0.0,
  0.0,
  4.5)]

In [15]:
# Filter the only the valid records using the predefined filter() function
taxilinesCorrected = testRDD.filter(correctRows)
taxilinesCorrected.take(1)

[('07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  datetime.datetime(2013, 1, 1, 0, 0),
  datetime.datetime(2013, 1, 1, 0, 2),
  120,
  0.44,
  -73.956528,
  40.716976,
  -73.96244,
  40.715008,
  'CSH',
  3.5,
  0.5,
  0.5,
  0.0,
  0.0,
  4.5)]

In [16]:


taxiMap = taxilinesCorrected.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16]))

taxiDriverPair = taxiMap.map(lambda x: (x[0],x[1])).collect()

medalliondrivepair = spark.sparkContext.parallelize(taxiDriverPair)

drivercountbytaxi = medalliondrivepair.groupByKey().mapValues(len)

drivercountbytaxitop10 = drivercountbytaxi.top(10, lambda x:x[1])

result_task1 = spark.sparkContext.parallelize(drivercountbytaxitop10)
result_task1.collect()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/pauljiang/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/pauljiang/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/pauljiang/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Exception in thread "serve RDD 35" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:655)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:631)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:588)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:546)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


23/01/28 03:46:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1015293 ms exceeds timeout 120000 ms
23/01/28 03:46:23 WARN SparkContext: Killing executors is not supported by current scheduler.


In [51]:
driverAvgMoney = taxiMap.map(lambda idx: (idx[1],(((float(idx[16]))/(float(idx[4]))*60),1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))
avgTop = driverAvgMoney.top(10, lambda x:x[1])
topRDD = spark.sparkContext.parallelize(avgTop)
topRDD.collect()


[('371910291B554B9821CA346094832F60', 7.0249999999999995),
 ('911299BE5F3FF1DFD6E3F4BFEACB6E26', 5.571428571428571),
 ('3C80E82705CE0E0A15D12ACD455C0B58', 4.535714285714286),
 ('8C25B80C13FE9A642D82EE6E5B157C39', 4.5),
 ('B72DEE45763A098CE0434638D9DAD7B0', 4.428571428571429),
 ('74D1BD03A75AF44B8293DB50F6943570', 4.355263157894737),
 ('90253C37D48D6BDCA8E0833804902306', 4.0675675675675675),
 ('08DB3F9FCF01530D6F7E70EB88C3AE5B', 3.833333333333333),
 ('8C72DB8F9EB81A2D4F923C909D93FFDB', 3.75),
 ('904DB583CC701A7BE6F483CDB2D470D5', 3.74375)]